In [60]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
# to divide train and test set
from sklearn.model_selection import train_test_split
from sklearn import linear_model


# feature scaling
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.model_selection import train_test_split


In [2]:
df = pd.read_csv('/Users/flatironschool/Documents/nyc-ds-010620/Projects/baseball_HR_linear_regression/Loading_Dataset/cleaned_data.csv', index_col=0)

In [3]:
df.columns

Index(['avg_hit_angle', 'avg_hit_speed', 'fbld', 'avg_hr_distance',
       'ev95percent', 'brl_pa', 'ba', 'est_ba', 'slg', 'est_slg', 'woba',
       'est_woba', 'Age', 'HR', 'OBP', 'OPS', 'HR_per_PA', 'RBI_per_PA',
       'SO_per_PA', 'SH_per_PA', 'SF_per_PA', 'IBB_per_PA', 'CHR_per_PA',
       'AS__2', 'AS__3', 'Tm__ATL', 'Tm__BAL', 'Tm__BOS', 'Tm__CHC', 'Tm__CHW',
       'Tm__CIN', 'Tm__CLE', 'Tm__COL', 'Tm__DET', 'Tm__HOU', 'Tm__KCR',
       'Tm__LAA', 'Tm__LAD', 'Tm__MIA', 'Tm__MIL', 'Tm__MIN', 'Tm__NYM',
       'Tm__NYY', 'Tm__OAK', 'Tm__PHI', 'Tm__PIT', 'Tm__SDP', 'Tm__SEA',
       'Tm__SFG', 'Tm__STL', 'Tm__TBR', 'Tm__TEX', 'Tm__TOR', 'Tm__TOT',
       'Tm__WSN'],
      dtype='object')

In [4]:
X = df.drop(['HR_per_PA'], axis = 1)
Y = df.HR_per_PA

In [5]:
# from sklearn.feature_selection import SelectKBest, f_regression,mutual_info_regression

# selector = SelectKBest(f_regression, k=11)

# selector.fit(X, Y)

In [6]:
# selected_columns = X.columns[selector.get_support()]
# removed_columns = X.columns[~selector.get_support()]

In [7]:
# selected_columns

In [8]:
# Let's separate into train and test set
# Remember to set the seed (random_state for this sklearn function)
X_train, X_test, y_train, y_test = train_test_split(df[['avg_hit_angle', 'avg_hit_speed', 'brl_pa', 'est_slg', 
                                                        'Age', 'AS__2', 'AS__3']], Y,
                                                    test_size=0.2,
                                                    random_state=0) # we are setting the seed here
X_train.shape, X_test.shape

((686, 7), (172, 7))

df[['avg_hit_angle', 'avg_hit_speed', 'ev95percent',
       'brl_pa', 'est_slg', 'est_woba', 'Age', 'OBP', 'AS__2', 'AS__3', 'RBI_per_PA']]

In [9]:
# building a linear regression model using statsmodel 
# from statsmodels.formula.api import ols

# indep_vars = '+'.join([var for var in df.columns if var != 'HR_per_PA'])
# formula = 'HR_per_PA' + "~" + indep_vars
# lr_model = ols(formula = formula, data = df).fit()

In [10]:
# lr_model.summary()

In [61]:
scale = StandardScaler()
X_train_transformed = scale.fit_transform(X_train)
X_test_transformed = scale.transform(X_test)

In [62]:
# Build a Ridge, Lasso and regular linear regression model  
# Note that in scikit-learn, the regularization parameter is denoted by alpha (and not lambda)
ridge = Ridge(alpha=0.95)
ridge.fit(X_train_transformed, y_train)

Ridge(alpha=0.95, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [63]:
lasso = Lasso(alpha=0.5)
lasso.fit(X_train_transformed, y_train)

Lasso(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [64]:
lin = LinearRegression()
lin.fit(X_train_transformed, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [65]:
# Generate preditions for training and test sets
y_h_ridge_train = ridge.predict(X_train_transformed)
y_h_ridge_test = ridge.predict(X_test_transformed)



In [66]:
y_h_lasso_train = lasso.predict(X_train_transformed)
y_h_lasso_test = lasso.predict(X_test_transformed)



In [67]:
y_h_lin_train = lin.predict(X_train_transformed)
y_h_lin_test = lin.predict(X_test_transformed)

In [68]:
print('Train Error Ridge Model', np.sum((y_train - y_h_ridge_train)**2))
print('Test Error Ridge Model', np.sum((y_test - y_h_ridge_test)**2))
print('\n')

print('Train Error Lasso Model', np.sum((y_train - y_h_lasso_train)**2))
print('Test Error Lasso Model', np.sum((y_test - y_h_lasso_test)**2))
print('\n')

print('Train Error Unpenalized Linear Model', np.sum((y_train - lin.predict(X_train_transformed))**2))
print('Test Error Unpenalized Linear Model', np.sum((y_test - lin.predict(X_test_transformed))**2))

Train Error Ridge Model 0.08386586806576354
Test Error Ridge Model 0.021733272869734276


Train Error Lasso Model 0.16860654108573525
Test Error Lasso Model 0.04550486368984545


Train Error Unpenalized Linear Model 0.083865333622162
Test Error Unpenalized Linear Model 0.02172910637508791


In [69]:
#import the metrics module from sklearn
from sklearn import metrics


In [70]:
print(np.sqrt(metrics.mean_squared_error(y_test, y_h_ridge_test))/y_test.std())


0.6890769246222196


In [71]:
alpha_list = np.linspace(.01, 1, 20)

In [72]:
for a in alpha_list:
    ridge = Ridge(alpha=a)
    ridge.fit(X_train_transformed, y_train)
    
    # Generate preditions for training and test sets
    y_h_ridge_train = ridge.predict(X_train_transformed)
    y_h_ridge_test = ridge.predict(X_test_transformed)

    
    print('Train Error Ridge Model with alpha: {}'.format(a), np.sum((y_train - y_h_ridge_train)**2))
    print('Test Error Ridge Model with alpha: {}'.format(a), np.sum((y_test - y_h_ridge_test)**2))
    print('\n')
    
        

Train Error Ridge Model with alpha: 0.01 0.08386533368251695
Test Error Ridge Model with alpha: 0.01 0.021729149215326662


Train Error Ridge Model with alpha: 0.06210526315789474 0.08386533594762163
Test Error Ridge Model with alpha: 0.06210526315789474 0.021729372792714335


Train Error Ridge Model with alpha: 0.11421052631578947 0.08386534147820385
Test Error Ridge Model with alpha: 0.11421052631578947 0.02172959696795794


Train Error Ridge Model with alpha: 0.16631578947368422 0.0838653502638885
Test Error Ridge Model with alpha: 0.16631578947368422 0.021729821739000704


Train Error Ridge Model with alpha: 0.21842105263157896 0.08386536229433872
Test Error Ridge Model with alpha: 0.21842105263157896 0.021730047103793428


Train Error Ridge Model with alpha: 0.2705263157894737 0.08386537755925591
Test Error Ridge Model with alpha: 0.2705263157894737 0.021730273060294465


Train Error Ridge Model with alpha: 0.32263157894736844 0.08386539604837936
Test Error Ridge Model with alpha:

In [92]:
# #testing alphas on lasso

# for a in alpha_list:
#     lasso = Lasso(alpha=a)
#     lasso.fit(X_train_transformed, y_train)
    
#     # Generate preditions for training and test sets
#     y_h_lasso_train = lasso.predict(X_train_transformed)
#     y_h_lasso_test = lasso.predict(X_test_transformed)

    
#     print('Train Error Lasso Model with alpha: {}'.format(a), np.sum((y_train - y_h_lasso_train)**2))
#     print('Test Error Lasso Model with alpha: {}'.format(a), np.sum((y_test - y_h_lasso_test)**2))
#     print('\n')
    
        

In [73]:
from sklearn.preprocessing import PolynomialFeatures

In [74]:
#test polynomial features

poly = PolynomialFeatures(interaction_only=False,include_bias = False)
poly.fit_transform(X_train_transformed)

array([[-0.41711912, -1.84123644, -0.24216056, ...,  0.70646766,
        -0.62052937,  0.54504505],
       [ 0.24891891, -0.02593291,  1.27263102, ...,  1.41549296,
        -1.61152727,  1.83471074],
       [-0.16448401, -0.67137416, -0.87332372, ...,  0.70646766,
        -0.62052937,  0.54504505],
       ...,
       [-2.71380198, -0.38899361, -1.16786653, ...,  1.41549296,
         0.87835495,  0.54504505],
       [ 0.01925062, -0.42933369,  0.09445979, ...,  1.41549296,
        -1.61152727,  1.83471074],
       [-0.71568789,  0.82120874, -0.2842381 , ...,  0.70646766,
        -0.62052937,  0.54504505]])

In [75]:

clf = linear_model.LinearRegression()
clf.fit(X_train_transformed, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [76]:
poly_predictions = clf.predict(X_test_transformed)

In [77]:
print(np.sqrt(metrics.mean_squared_error(y_test, poly_predictions))/y_test.std())

0.6890108698464478


In [78]:
print(poly_predictions*500-y_test*500)

144     0.176202
119   -13.752907
47      0.374618
133    -5.547582
192   -16.391426
         ...    
157    -6.671928
171    -0.686641
198     9.038365
50      4.035007
168     2.643126
Name: HR_per_PA, Length: 172, dtype: float64


In [79]:
ridge_poly = Ridge(alpha=.95)

ridge_poly.fit(X_train_transformed, y_train)

Ridge(alpha=0.95, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [80]:
poly_ridge_pred = ridge_poly.predict(X_test_transformed)

In [81]:
print(np.sqrt(metrics.mean_squared_error(y_test, poly_ridge_pred))/y_test.std())

0.6890769246222196


In [82]:
for a in alpha_list:
    ridge_poly = Ridge(alpha=a)
    ridge_poly.fit(X_train_transformed, y_train)
    
    # Generate preditions for training and test sets
    y_h_poly_ridge_train = ridge_poly.predict(X_train_transformed)
    y_h_poly_ridge_test = ridge_poly.predict(X_test_transformed)

    
    print('Train Error Poly Ridge Model with alpha: {}'.format(a), np.sum((y_train - y_h_poly_ridge_train)**2))
    print('Test Error Poly Ridge Model with alpha: {}'.format(a), np.sum((y_test - y_h_poly_ridge_test)**2))
    print('\n')

Train Error Poly Ridge Model with alpha: 0.01 0.08386533368251695
Test Error Poly Ridge Model with alpha: 0.01 0.021729149215326662


Train Error Poly Ridge Model with alpha: 0.06210526315789474 0.08386533594762163
Test Error Poly Ridge Model with alpha: 0.06210526315789474 0.021729372792714335


Train Error Poly Ridge Model with alpha: 0.11421052631578947 0.08386534147820385
Test Error Poly Ridge Model with alpha: 0.11421052631578947 0.02172959696795794


Train Error Poly Ridge Model with alpha: 0.16631578947368422 0.0838653502638885
Test Error Poly Ridge Model with alpha: 0.16631578947368422 0.021729821739000704


Train Error Poly Ridge Model with alpha: 0.21842105263157896 0.08386536229433872
Test Error Poly Ridge Model with alpha: 0.21842105263157896 0.021730047103793428


Train Error Poly Ridge Model with alpha: 0.2705263157894737 0.08386537755925591
Test Error Poly Ridge Model with alpha: 0.2705263157894737 0.021730273060294465


Train Error Poly Ridge Model with alpha: 0.3226315

In [83]:
ridge_poly_1 = Ridge(alpha=0.01)

ridge_poly_1.fit(X_train_transformed, y_train)

Ridge(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [84]:
poly_ridge_1_pred = ridge_poly_1.predict(X_test_transformed)

In [85]:
print(np.sqrt(metrics.mean_squared_error(y_test, poly_ridge_1_pred))/y_test.std())

0.6890115490592765


### Recursive Feature Elimination

In [86]:


from sklearn.feature_selection import RFECV

ols = linear_model.LinearRegression()

In [87]:
# Create recursive feature eliminator that scores features by mean squared errors
selector = RFECV(estimator=ols, step=1, cv=5, scoring='neg_mean_squared_error')

# Fit recursive feature eliminator 
selector.fit(X_train_transformed, y_train)

RFECV(cv=5,
      estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                                 normalize=False),
      min_features_to_select=1, n_jobs=None, scoring='neg_mean_squared_error',
      step=1, verbose=0)

In [88]:
selected_columns = X_train.columns[selector.support_]
removed_columns = X_train.columns[~selector.support_]


In [89]:
lm69 = linear_model.LinearRegression()
lm69.fit(X_train_transformed, y_train)
y_train_pred = lm69.predict(X_train_transformed)
y_test_pred = lm69.predict(X_test_transformed)

In [90]:
print(np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))/y_test.std())


0.6890108698464478
